<a href="https://colab.research.google.com/github/ttussing/US-Gun-Exploration-Visualization/blob/master/US_Gun_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Things to do 
#Describe how we gathered and cleaned the data 
#2-3 datasets 
#Feature Engineering 
#Describe features 
#Descriptive statistics 
#Describe why we chose the data? 

In [0]:
import pandas as pd
import numpy as np
from google.colab import files

In [0]:
world_url = 'https://raw.githubusercontent.com/ttussing/US-Gun-Exploration-Visualization/master/World_firearms.csv'
state_url = 'https://raw.githubusercontent.com/ttussing/US-Gun-Exploration-Visualization/master/Firearm%20Deaths%20Per%20State%20Per%20Year/All_Firearm_Deaths_State_Year.csv'
ms_url = 'https://raw.githubusercontent.com/ttussing/US-Gun-Exploration-Visualization/master/Mass%20Shooting/Mass_Shooting_Data_2013-2019.csv'
df_world = pd.read_csv(world_url,index_col=0)
df_state = pd.read_csv(state_url,index_col=0)
df_ms = pd.read_csv(ms_url,index_col=0)

## World Firearms Data

[World Data Set](https://docs.google.com/spreadsheets/d/1chqUZHuY6cXYrRYkuE0uwXisGaYvr7durZHJhpLGycs/edit#gid=0)

[Article Describing the Data Set](https://www.theguardian.com/news/datablog/2012/jul/22/gun-homicides-ownership-world-list#data)

**Description of the Dataset:**

- A collection of data from the UNODC  
Number of firearms by 

- The world’s crime figures are collected by the UNODC through its annual crime survey

- Data is missing for: Russia, China and Afghanistan


**Takeaways from this Dataset:**

- US leads on all gun ownership stats

- Centra/South America leads on all murder stats, far ahead of US
is ahead in murder only when filtering for high HDI countries

In [26]:
df_world.head()

,name_semicolon_delimited,killed,wounded,city,state,sources_semicolon_delimited
date,,,,,,
5/15/19,Unkown (victims included a 5-yr-old girl),0,4,Los Angeles,CA,https://abc7.com/3-men-5-year-old-girl-shot-at...
5/13/19,Unknown,4,1,St. Louis,MO,https://www.stltoday.com/news/local/crime-and-...
5/13/19,Unknown (victims included 1 juvenile),0,4,New Orleans,LA,https://www.wdsu.com/article/four-injured-in-c...
5/12/19,Unknown,0,4,Paulsboro,NJ,https://6abc.com/5-hurt-after-gunmen-open-fire...
5/11/19,Thomas Modzel,0,4,Effort,PA,https://wnep.com/2019/05/11/four-people-shot-a...


In [27]:
df_state.head()

,Sex,Race,State,Ethnicity,Age Group,First Year,Last Year,Cause of Death,Year,Deaths,Population,Crude Rate,Age-Adjusted Rate
Type,,,,,,,,,,,,,
Unitentional,Both Sexes,All Races,Alabama,Both,All Ages,1999,2017,Unintentional Firearm,1999.0,51,4430143,1.15,1.15722772
Unitentional,Both Sexes,All Races,Alabama,Both,All Ages,1999,2017,Unintentional Firearm,2000.0,34,4451687,0.76,0.76222845
Unitentional,Both Sexes,All Races,Alabama,Both,All Ages,1999,2017,Unintentional Firearm,2001.0,41,4467634,0.92,0.926071313
Unitentional,Both Sexes,All Races,Alabama,Both,All Ages,1999,2017,Unintentional Firearm,2002.0,45,4480089,1,0.997164737
Unitentional,Both Sexes,All Races,Alabama,Both,All Ages,1999,2017,Unintentional Firearm,2003.0,38,4503491,0.84,0.82537211


In [28]:
df_ms.head()

,name_semicolon_delimited,killed,wounded,city,state,sources_semicolon_delimited
date,,,,,,
5/15/19,Unkown (victims included a 5-yr-old girl),0,4,Los Angeles,CA,https://abc7.com/3-men-5-year-old-girl-shot-at...
5/13/19,Unknown,4,1,St. Louis,MO,https://www.stltoday.com/news/local/crime-and-...
5/13/19,Unknown (victims included 1 juvenile),0,4,New Orleans,LA,https://www.wdsu.com/article/four-injured-in-c...
5/12/19,Unknown,0,4,Paulsboro,NJ,https://6abc.com/5-hurt-after-gunmen-open-fire...
5/11/19,Thomas Modzel,0,4,Effort,PA,https://wnep.com/2019/05/11/four-people-shot-a...
